In [ ]:
# UDP Broadcast Client using IPython Background Jobs (non-blocking)
# Run this in a separate Jupyter notebook

import socket
import json
import datetime
import uuid
import sys
from IPython.display import clear_output

# Create a unique ID for this client
client_id = str(uuid.uuid4())[:8]

# Store received messages for demonstration purposes
received_messages = []
last_processed_id = -1

# Define the port clients will listen on
client_port = 37020

# Create and configure socket directly (no asyncio)
client_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM, socket.IPPROTO_UDP)
client_socket.setsockopt(socket.SOL_SOCKET, socket.SO_BROADCAST, 1)
client_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)

# Make socket non-blocking
client_socket.setblocking(False)

# Bind to client port
client_socket.bind(('0.0.0.0', client_port))

# Socket for responses
response_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

# Print initialization message
print(f"Client {client_id} started, listening on port {client_port}")
print("To stop the client, restart the kernel or interrupt the cell")

# Function to process data
def process_message(data, addr):
    global last_processed_id
    
    try:
        server_ip, server_port = addr
        
        # Decode and parse the message
        message_str = data.decode('utf-8')
        message = json.loads(message_str)
        
        # Extract timestamp and state
        timestamp = message.get("timestamp", "unknown")
        state = message.get("state", "unknown")
        message_id = message.get("message_id", -1)
        
        # Only process new messages (in case of duplicates)
        if message_id > last_processed_id:
            last_processed_id = message_id
            
            # Get current time
            receive_time = datetime.datetime.now().isoformat()
            
            # Clear output to avoid long scrolling
            clear_output(wait=True)
            
            # Print client info again
            print(f"Client {client_id} listening on port {client_port}")
            print("To stop the client, restart the kernel or interrupt the cell")
            
            # Print received message
            print(f"\nReceived broadcast from {server_ip}:{server_port}")
            print(f"Message ID: {message_id}")
            print(f"Timestamp: {timestamp}")
            print(f"State: {state}")
            
            # Append to message history
            received_messages.append({
                "server_ip": server_ip,
                "server_port": server_port,
                "timestamp": timestamp,
                "receive_time": receive_time,
                "state": state,
                "message_id": message_id
            })
            
            # Send response back to the server
            response = f"Client {client_id} received message {message_id}"
            response_socket.sendto(response.encode('utf-8'), (server_ip, server_port))
            
            # Print statistics
            print(f"Total messages received: {len(received_messages)}")
            
            # Display the last 5 states for demonstration
            if len(received_messages) >= 5:
                recent_states = [msg["state"] for msg in received_messages[-5:]]
                print(f"Last 5 states: {recent_states}")
    
    except json.JSONDecodeError:
        print(f"Error: Received invalid JSON data")
    
    except Exception as e:
        print(f"Error: {e}")

# Import the necessary IPython components for the background task
from IPython.lib.backgroundjobs import BackgroundJobManager
from threading import Thread
import time

# Create a background job manager
jobs = BackgroundJobManager()

# Function to check for messages in a background thread
def check_for_messages():
    try:
        while True:
            try:
                # Try to receive data
                data, addr = client_socket.recvfrom(1024)
                # Process the message
                process_message(data, addr)
            except BlockingIOError:
                # No data available, wait a bit and try again
                time.sleep(0.1)
            except Exception as e:
                print(f"Error in receive loop: {e}")
                time.sleep(1)  # Wait a bit longer on error
    except KeyboardInterrupt:
        print("Client stopping...")
    finally:
        print("Client stopped.")

# Start the background job
jobs.new(check_for_messages)

print(f"Client {client_id} is now running in the background.")
print("Messages will appear here when received.")

# To stop the client, you need to restart the kernel or use jobs.flush()